In [1]:
import os

import pandas as pd

os.chdir('/homes/bussotti/XFC2/code')

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from explainable_fact_checking.charts.notebook_utility import *
import nltk
from rank_bm25 import BM25Okapi

/home/bussotti/.conda/envs/feverous2/bin/python3


# LOAD

In [3]:
from explainable_fact_checking.dataset_preprocessing.pre import preprocess_AVERITEC, convert_qa_dict_2txt, qa2txt
from explainable_fact_checking.datasets_loaders import load_std_dataset
from explainable_fact_checking.datasets_loaders import RobertaTokenizerWrapper

In [4]:
from huggingface_hub import hf_hub_download

ds_path = os.path.join(xfc.experiment_definitions.C.DATASET_DIR_V2, 'AVERITEC')
preprocessed_path = os.path.join(ds_path, 'preprocessed')
os.makedirs(preprocessed_path, exist_ok=True)
files = ['train', 'dev', 'test']
files_path = {}
for f in files:
    files_path[f] = hf_hub_download(repo_id="chenxwh/AVeriTeC", filename='data/' + f + '.json', cache_dir=ds_path,
                                    revision='7f83bddcd8f16570446c51578ab3ddf6a9d350b1')

In [5]:
datasets = {}
for f in files:
    datasets[f] = json.load(open(files_path[f]))

In [6]:
e0 = datasets['train'][0]
e0

{'claim': 'Hunter Biden had no experience in Ukraine or in the energy sector when he joined the board of Burisma.',
 'required_reannotation': False,
 'label': 'Supported',
 'justification': 'No former experience stated.',
 'claim_date': '25-8-2020',
 'speaker': 'Pam Bondi',
 'original_claim_url': None,
 'fact_checking_article': 'https://web.archive.org/web/20210111003633/https://www.politifact.com/article/2020/aug/26/fact-checking-second-night-2020-rnc/',
 'reporting_source': 'Speech at The Republican National Convention',
 'location_ISO_code': 'US',
 'claim_types': ['Position Statement'],
 'fact_checking_strategies': ['Written Evidence'],
 'questions': [{'question': 'Did Hunter Biden have any experience in the energy sector at the time he joined the board of the  Burisma energy company in 2014',
   'answers': [{'answer': 'No',
     'answer_type': 'Boolean',
     'source_url': 'https://en.wikipedia.org/wiki/Hunter_Biden',
     'source_medium': 'Web text',
     'boolean_explanation': "H

# Preprocessing

In [7]:
question_answer_list = [x['questions'] for dataset in [datasets['train'], datasets['dev']] for x in dataset]
answer_distinct_fields = set()
for question_answer in question_answer_list:
    for qa in question_answer:
        for answer in qa['answers']:
            answer_distinct_fields.update(set(answer.keys()))

In [8]:
for prepend_question in [False, True]:
    for dataset in [datasets['train'], datasets['dev'], datasets['test']]:
        preprocess_AVERITEC(dataset, prepend_question=prepend_question)
    for f in files:
        t_name = f + ('_no_question' if prepend_question is False else '')
        with open(os.path.join(preprocessed_path, t_name + '.json'), 'w') as f_out:
            json.dump(datasets[f], f_out)

In [9]:
datasets_converted = {}
for f in ['train', 'dev']:
    datasets_converted[f] = load_std_dataset(preprocessed_path, f + '.json')

Map:   0%|          | 0/3068 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3068 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
pd.Series([len(e['evidence']) for e in datasets_converted['dev']]).value_counts()

2    174
1    119
3     96
4     49
5     23
6      8
7      6
Name: count, dtype: int64

In [11]:
len(datasets_converted['dev'])

475

In [12]:
pd.Series([len(e['evidence']) for e in datasets['dev']]).value_counts()

2     177
1     120
3     101
4      52
5      25
6      11
7       9
9       2
8       1
10      1
12      1
Name: count, dtype: int64

In [13]:
# get average number of evidence texts
sum([len(e['evidence']) for e in datasets['train']]) / len(datasets['train'])

2.6026727509778356

In [14]:
sum([len(e['evidence']) for e in datasets['dev']]) / len(datasets['dev'])

2.574

In [15]:
# load datasets with questions
datasets = {}
for f in files:
    t_name = f
    with open(os.path.join(preprocessed_path, t_name + '.json'), 'r') as f_in:
        datasets[f] = json.load(f_in)

## 

# Noisy evidence

Gather all retreived evidence QA form with id of their claim.

Find actual token length of claim and evidence, and the remaining tokens for noise.

Find candidate noisy evidence from retrieved evidence.

Filter retrieved evidence with same source_url of gold evidence.

Sort by BM25 similarity score, with equal score, sort by token length.

Look at averitec.py for more details.